### Spiel-Komponente ruft selber update-Funktion der Darstellungskomponente auf

Das Modul `schach_1.py` unterscheidet sich von `schach_0.py`
durch eine kleine, aber konzeptuell wichtige Änderung. 
Es gibt nun eine Funktion `update(event, **kwargs)`,
die im Moment nur ihre Argumente ausgibt, und
jeweils am Ende der Funktionen `move` und `new_game` aufgerufen wird.
Das 2. Argument enthält dabei Information, die ein bequemes Anpassen der Darstellung des Spielzustandes erlauben.

```python
def update(event, **kwargs):
    print(f'event: {event}, kwargs: {kwargs}')

def move(src, target):
    ...
    changes = ((SPACE, *src), (char, *target))
    update('move', changes=changes)


def new_game():
    set_startpos()
    update('new_game', changes=get_pieces())
```

Wir überschreiben dann die Funktion `update`
mit einer update-Funktion der Darstellungskomponente `schach_D1.py` der gleichen Signatur.



```python
def update_D(canvas, event, **kwargs):
    if event == 'new_game':
        canvas.clear()
        apply_changes(canvas, kwargs['changes'])
    if event == 'move':
        apply_changes(canvas, kwargs['changes'])

# update der Spielkomponente überschreiben
update = lambda event, **kwargs: update_D(canvas, event, **kwargs)        
```

Dadurch veranlasst die Spielkomponente selber das Update der graphischen Darstellung!

In [8]:
import schach_1 as schach
import schach_D1 as D
from ipycanvas import MultiCanvas
from time import sleep


layout = {'border': '1px solid black'}
mcanvas = MultiCanvas(2, width=200, height=200, layout=layout)
bg, fg = mcanvas

D.draw_chessboard(bg)

# update der Spielkomponente wird ueberschreiben
schach.update = lambda event, **kwargs: D.update(fg, event, **kwargs)
schach.new_game()
mcanvas

MultiCanvas(height=200, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_r…

In [ ]:
moves = [('e2', 'e4'), ('e7', 'e5'), ('f1', 'c4'), ('f8', 'c5'),
         ('d1', 'h5'), ('g8', 'f6'), ('h5', 'f7')]

schach.new_game()
for src, target in moves:
    schach.move(src, target)
    sleep(1)

In [ ]:
schach.new_game()

In [ ]:
schach.move('g1', 'f3')

### Aufgaben
1. Definiere im File `schach_D1.py` einen Dict, der
    den Buchstaben `'KDTLSBkdtlsb'` die Symbole `'♔♕♖♗♘♙♚♛♜♝♞♟'`
zuordnet. Modifiziere dann die Funktion `apply_changes` so, dass jeweils das Symbol statt des Buchstabens verwendet wird.

2. Implementiere in der nachstehenden Zelle die Funktionen `on_mouse_down(x, y)` und
`on_mouse_up(x, y)` so, dass sich die Figuren mit der Maus bewegen lassen.  
 `on_mouse_down(x, y)` speichert lediglich `(x, y)` im Dict `state`.
`on_mouse_up(x, y)` soll die Funktion `xy2cr(x, y, board_spec)` aus `helpers` verwenden, um die geklickten Positionen in
Tuples `(Spalte, Reihe)` umzurechnen und dann `schach.raw_move` aufzurufen.

In [ ]:
import schach_1 as schach
import schach_D1 as D
from ipycanvas import MultiCanvas
from ipywidgets import Output
import helpers as H
from IPython.display import display


state = {'mouse_down': None}

layout = {'border': '1px solid black'}
out = Output(layout=layout)
mcanvas = MultiCanvas(2, width=200, height=200, layout=layout)
bg, fg = mcanvas


@out.capture(clear_output=True)
def on_mouse_down(x, y):
    ...
    print(f'state[\'mouse_down\'] set to ({x}, {y})')


@out.capture()
def on_mouse_up(x, y):
    ...
    print(f'move piece from {state['mouse_down']} to {(x, y)}')


D.draw_chessboard(bg)

# update der Spielkomponente wird ueberschreiben
schach.update = lambda event, **kwargs: D.update(fg, event, **kwargs)
schach.new_game()

mcanvas.on_mouse_down(on_mouse_down)
mcanvas.on_mouse_up(on_mouse_up)

display(mcanvas, out)